# Transform images to sketches using diferent methods

In [1]:
import shutil, gc, math, random
from pathlib import Path
from PIL import Image
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToPILImage, ToTensor, Normalize, Compose, Pad
import fastai.basics as faib
import fastai.vision.all as fv
from fastprogress.fastprogress import progress_bar

from sketch_keras import SketchKeras
from sketch_simplification import SimplifyModel
from anime2sketch import A2SModel
from xdog import XDOG

to_image = ToPILImage()
to_tensor = ToTensor()

## Sketch Keras

In [2]:
def resize_pad_tensor(img, size):
    h, w = img.shape
    nsize = [size, size]
    if h>w:
        nsize[1] = int(size / h * w)
    else:
        nsize[0] = int(size / w * h)
    tens = to_tensor(img.resize(nsize))
    tens = Pad((0, 0, 512-tens.shape[2], 512-tens.shape[1]))(tens)
    return tens, nsize[::-1]

In [3]:
keras_model = SketchKeras(25,3,0).cuda().eval()

In [5]:
def keras_to_lineart(imgfiles, thresh = 0.0):
    global save_folder
    if isinstance(imgfiles, (str, Path)):
        imgfiles = [imgfiles]
    
    imgfiles = [Path(i) for i in imgfiles]
    
    tensors, sizes, nsizes = [], [], []
    for f in imgfiles:
        im = Image.open(f).convert("RGB")
        sizes.append(im.shape)
        t, ns = resize_pad_tensor(im, 512)
        tensors.append(t)
        nsizes.append(ns)
    
    x = torch.stack(tensors).cuda()
    with torch.no_grad():
        pred = keras_model(x).cpu()
    
    for p, ns, siz, f in zip(pred, nsizes, sizes, imgfiles):
        outp = to_image(p[:,:ns[0],:ns[1]]).resize(siz[::-1])
        outp.save(str(save_folder/f.name))

## XDOG

In [6]:
def open_bw_tensor(fname):
    return to_tensor(Image.open(fname).convert('L'))

In [7]:
xdog_model = XDOG(4.5, 0.4, 0.95, -0.7, 1e9, 25).cuda().eval()

In [8]:
def xdog_to_lineart(imgfiles):
    global save_folder
    if isinstance(imgfiles, (str, Path)):
        imgfiles = [imgfiles]
    
    imgfiles = [Path(i) for i in imgfiles]
    x = torch.stack(list(map(open_bw_tensor, imgfiles))).cuda()
    
    with torch.no_grad():
        x = xdog_model(x).cpu()
    
    for p, f in zip(x,  imgfiles):
        outp = to_image(p)
        outp.save(str(save_folder/f.name))

## Simplification

In [5]:
simplify_model = SimplifyModel().cuda().eval()

In [10]:
def simp_to_lineart(imgfiles, immean=0.9664114577640158 , imstd=0.0858381272736797):
    global save_folder
    if isinstance(imgfiles, (str, Path)):
        imgfiles = [imgfiles]
    imgfiles = [Path(i) for i in imgfiles]
    imgs = torch.stack(list(map(open_bw_tensor, imgfiles))).cuda()
    
    with torch.no_grad():
        out = simplify_model(imgs).cpu()
        pil_out = [to_image(o) for o in out]
    
    for name, i_out in zip(imgfiles, pil_out):
        i_out.save(str(save_folder/name.name))

## Anime2Sketch

In [4]:
a2s_model = A2SModel().eval().cuda()

In [15]:
def a2s_to_lineart(imgfiles):
    global save_folder
    if isinstance(imgfiles, (str, Path)):
        imgfiles = [imgfiles]
    imgfiles = [Path(i) for i in imgfiles]
    
    imgs = [Image.open(f).convert("RGB") for f in imgfiles]
    
    tensors, sizes, nsizes = [], [], []
    for im in imgs:
        sizes.append(im.shape)
        t, ns = resize_pad_tensor(im, 512)
        tensors.append(t)
        nsizes.append(ns)
    
    x = torch.stack(tensors).cuda()
    with torch.no_grad():
        pred = a2s_model(x).cpu()
        
    for p, ns, siz, f in zip(pred, nsizes, sizes, imgfiles):
        outp = to_image(p[:,:ns[0],:ns[1]]).resize(siz[::-1])
        outp.save(str(save_folder/f.stem)+".jpg")

In [5]:
def ensamble(imgfiles):
    global save_folder
    if isinstance(imgfiles, (str, Path)):
        imgfiles = [imgfiles]
    imgfiles = [Path(i) for i in imgfiles]
    
    imgs = [Image.open(f).convert("RGB") for f in imgfiles]
    
    tensors, sizes, nsizes = [], [], []
    for im in imgs:
        sizes.append(im.shape)
        t, ns = resize_pad_tensor(im, 512)
        tensors.append(t)
        nsizes.append(ns)
    
    x = torch.stack(tensors).cuda()
    gc.collect()
    torch.cuda.empty_cache()
    with torch.no_grad():
        w = random.random()
        pred = a2s_model(x)*w + keras_model(x)*(1-w)
        pred = pred.cpu()
        
    for p, ns, siz, f in zip(pred, nsizes, sizes, imgfiles):
        outp = to_image(p[:,:ns[0],:ns[1]]).resize(siz[::-1])
        outp.save(str(save_folder/f.name))

### WORK

In [6]:
def chunks(iterable, n):
    curr = []
    for it in iterable:
        curr.append(it)
        if len(curr) == n:
            yield curr[:]
            curr = []
    if curr:
        yield curr

In [8]:
files = fv.get_image_files("path/to/images")
save_folder = Path("where/to/save")
save_folder.mkdir(exist_ok=True)

In [ ]:
for fs in progress_bar(list(chunks(files, 16))):
    ensamble(fs)